In [1]:
from data_generation_framework import *
import numpy as np

In [2]:
simulated_data = created_simulated_data_from_random_covariates()
simulated_data.head()

(1000, 21)


,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X16,X17,X18,X19,NOISE,T,Y0,Y1,Y,treatment_effect
431,-6.490447,0.782064,16.815941,-2.547431,1.183544,4.450439,-5.049501,-4.140983,2.213235,-3.028048,...,7.686774,2.158232,11.494431,5.051245,-0.316,1,0.548384,2.246512,2.246512,1.698128
667,-2.766957,8.313286,-9.518069,-9.914140,-1.258200,5.663301,13.507559,-0.457900,2.849851,9.659813,...,-0.844424,2.884125,4.047240,-7.228608,0.564,1,0.144910,1.489926,1.489926,1.345015
383,-6.919845,2.364335,1.512260,-5.242408,-4.506361,1.204675,2.638711,-3.414484,6.405304,-2.501685,...,2.589264,5.730961,-1.984678,2.329414,-2.471,1,-3.615033,-3.183212,-3.183212,0.431821
140,-2.951826,-1.370260,1.546239,5.058279,-0.562841,-2.900711,-4.993263,-0.263955,-1.456679,5.449712,...,0.879098,-0.554384,-6.385716,6.209115,0.825,1,0.403064,1.407053,1.407053,1.003989
920,0.124517,1.554645,-4.300951,3.453924,-2.532225,-6.510456,-2.615892,-1.822663,1.508804,3.842307,...,2.972999,-9.452384,-7.034369,1.393660,1.708,1,1.288426,2.224469,2.224469,0.936043
332,5.763751,6.350272,-7.643637,-4.390034,3.212220,-4.129070,-4.135847,2.892729,3.324329,8.675400,...,-3.823553,8.831324,12.289490,2.872609,-0.482,0,-0.902170,1.230346,-0.902170,2.132516
127,-1.849607,8.865462,-13.388939,-3.441065,0.191064,2.756565,-9.196510,7.534190,6.902443,6.935014,...,-6.930806,5.375405,2.861575,-4.121790,-0.526,1,-0.686705,0.513207,0.513207,1.199912
159,1.324780,-5.624006,-5.591169,1.055494,1.004751,2.485769,-0.859422,0.576052,-2.539105,4.184118,...,-0.466835,1.884188,1.430815,-5.960086,4.102,0,4.183823,6.265746,4.183823,2.081923
497,3.424983,-14.048957,1.717045,11.976446,2.901530,0.678343,2.589792,-0.979356,-4.942352,0.490219,...,-5.207430,-3.967251,-6.230587,6.860845,-1.006,1,-1.110917,0.165331,0.165331,1.276249
847,-3.910773,1.641161,3.527914,-0.409049,-0.347080,8.954245,-1.144754,-5.079019,-4.247032,-0.152946,...,4.572149,1.407569,-5.544304,2.359189,-0.848,1,-1.687010,-1.939946,-1.939946,-0.252936
